In [27]:
import os
import pandas as pd
import numpy as np
from numpy import sort
from os import listdir
from os.path import isfile, join

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt
import plotly.express as px

from IPython.display import display_html

from matplotlib_venn import venn3
from Project.Utils.norm import norm
from Project.Utils.max_corr import max_corr

from Project.Utils.visualize import search, searchTimeSeries, get_years

import warnings
warnings.filterwarnings("ignore")

In [28]:
data_path = os.getcwd() + '/Databases/'
output_path = os.getcwd() + '/Output/'
country_path = output_path + '/Country/'
region_path = output_path + '/Region/'
cluster_path = output_path + '/Cluster/'

In [29]:
file_regions = 'AuxiliarData/world-regions-mod.csv'

file_gold = 'GoldDataframe.csv'
file_corr_pearson = 'Corr_DF_Pearson.csv'
file_corr_spearman = 'Corr_DF_Spearman.csv'
file_agg_region = 'AggregatedRegion_DataFrame.csv'
file_agg_world = 'AggregatedWorld_DataFrame.csv'
file_norm_df = 'Norm_DF.csv'
file_shifted_corr_country = 'Shifted_Corr_Country.csv'
file_shifted_corr_region = 'Shifted_Corr_Region.csv'

In [30]:
col_country = 'Country'
col_year = 'Year'
col_region = 'Region'
col_gdp = 'GDP'
col_cluster = 'Cluster'
col_shift = 'Shift'

col_1comp = '1st_component'
col_2comp = '2nd_component'

In [31]:
# READ GOLD DATAFRME
# Read Golden Dataframe and initialize the variables that depend on it.

df = pd.read_csv(output_path + file_gold, on_bad_lines = 'warn', index_col = ['Region', 'Country', 'Year'])

# List of countries and list of regions.
country_list = list(np.sort(df.index.get_level_values(col_country).unique()))
region_list = list(np.sort(df.index.get_level_values(col_region).unique()))

mode_dict = {}
mode_dict[col_country] = country_list
mode_dict[col_region] = region_list
mode_list = [col_country, col_region]

# Range of years.
min_year = df.index.get_level_values(col_year).min()
max_year = df.index.get_level_values(col_year).max()

# List of all indicators, except for the GDP.
indicators_list = df.columns.tolist()
indicators_list.remove(col_gdp)
indicators_list.sort()

In [32]:
# READ CLUSTERS
# Read both Dataframes for each indicators group: the one with the indicator values, and the one with the components.

ind_dict = {}
cluster_dict = {}
cluster_list = []
comp_dict = {}

for element in listdir(cluster_path):
    url = join(cluster_path, element)
    if isfile(url) and url.endswith('_Comp.csv'):
        ind = element.removesuffix('_Comp.csv')
        comp_df = pd.read_csv(url, on_bad_lines = 'warn', index_col = [col_cluster, col_country])
        comp_dict[ind] = comp_df
        cluster_number_df = comp_df.drop(columns = comp_df.columns).reset_index(col_cluster, drop = False).rename({col_cluster: ind}, axis = 1)
        cluster_list.append(cluster_number_df)

    elif isfile(url) and url.endswith('.csv'):
        ind = element.removesuffix('.csv')
        ind_df = pd.read_csv(url, on_bad_lines = 'warn', index_col = [col_cluster, col_country])
        cluster_dict[ind] = ind_df.reset_index(col_cluster, drop = False)
        ind_dict[ind] = ind_df.columns

cluster_df = pd.concat(cluster_list, axis = 1)
cluster_df = cluster_df.fillna(-1).astype(int)

In [33]:
cluster_df

,All indicators,Economic Indicators,Equality Indicators,Social-demographic Indicators
Country,,,,
Albania,0,6,4,3
Algeria,0,7,6,9
Armenia,0,1,8,3
Bosnia and Herzegovina,0,1,0,3
Bulgaria,0,2,0,3
...,...,...,...,...
Sao Tome and Principe,9,1,8,5
Solomon Islands,9,6,8,5
Togo,9,4,8,4


In [34]:
class WidgetGlobalStatus:
    def __init__(self):

        self.mode = None #mode_list[0]
        self.zone = None
        self.ind = None
        self.ind_group = None
        self.df = pd.read_csv(output_path + file_gold, index_col = [col_country, col_year, col_region])
        self.corr_df = pd.read_csv(output_path + file_corr_spearman, index_col = [col_country])
        self.norm_df = pd.read_csv(output_path + file_norm_df, index_col = [col_country, col_region, col_year])
        self.cluster_df = cluster_df
        self.comp_df = None
        self.ind_df = None
        self.shift_df = None
        self.data = None
        self.years = None #range(min_year, max_year + 1)
        self.shift = None #0
        self.threshold = None
        self.pval = None

status_widget = WidgetGlobalStatus()

In [35]:
def group_of(ind: str):     # NEEDS ACCESS TO IND_DICT AND A LIST OF ALL INDICATORS
    for group in ind_dict.keys():
        if (len(ind_dict[group]) < len(indicators_list)) and (ind in ind_dict[group]):
            return group
    return None
    return [group if (ind in ind_dict[group]) else None for group in ind_dict.keys()]

# FIND A GOOD WAY TO AVOID RETURNING THE ALL INDICATORS GROUP

In [36]:
#plot_cluster_map_out = widgets.Output(layout={'border': '1px solid black'})
plot_cluster_map_out = widgets.Output()
table_cluster_out = widgets.Output(layout={'border': '1px solid black'})
venn_group_out = widgets.Output(layout={'border': '1px solid black'})
chart_shift_ind_out = widgets.Output(layout={'border': '1px solid black'})
chart_shift_corr_out = widgets.Output(layout={'border': '1px solid black'})






globe_corr_out = widgets.Output(layout={'border': '1px solid black'})
table_max_corr_country_out = widgets.Output(layout={'border': '1px solid black'})
time_window_out = widgets.Output(layout={'border': '1px solid black'})
table_high_out = widgets.Output(layout={'border': '1px solid black'})
plot_corr_ind_out = widgets.Output(layout={'border': '1px solid black'})

# CLUSTER MAP

In [37]:
plot_cluster_map_out

Output()

# CLUSTER TABLE

In [38]:
table_cluster_out

Output(layout=Layout(border='1px solid black'))

# CLUSTER VENN DIAGRAM

In [39]:
venn_group_out

Output(layout=Layout(border='1px solid black'))

# CHART OF INDICATORS VALUES

In [40]:
chart_shift_ind_out

Output(layout=Layout(border='1px solid black'))

# CHART OF INDICATORS CORRELATIONS

In [41]:
chart_shift_corr_out

Output(layout=Layout(border='1px solid black'))

In [42]:
globe_corr_out

Output(layout=Layout(border='1px solid black'))

In [43]:
table_max_corr_country_out

Output(layout=Layout(border='1px solid black'))

In [44]:
time_window_out

Output(layout=Layout(border='1px solid black'))

In [45]:
table_high_out

Output(layout=Layout(border='1px solid black'))

In [46]:
plot_corr_ind_out

Output(layout=Layout(border='1px solid black'))

In [47]:
@plot_cluster_map_out.capture(clear_output=True, wait=True)
def plot_cluster_map (df: pd.DataFrame, group_name: str = 'All indicators', *, col_1comp: str = '1st_component', col_2comp: str = '2nd_component'):
    if df is None: return None
    fig = px.scatter(df, x = col_1comp, y = col_2comp, text = df.index, size_max=100, color=col_cluster, category_orders={col_cluster: np.sort(df.loc[:, col_cluster].unique())})
    fig.update_layout(title_text = group_name, title_x=0.5)
    fig.update_traces(textposition = 'top center')
    fig.show()

@table_cluster_out.capture(clear_output=True, wait=True)
def table_cluster (df: pd.DataFrame, group_name: str = 'All indicators', country: str = '', cluster_number: int = 0):
    if df is None: return None
    try:
        df_s = df.style
        df_s.apply_index(lambda i: ['background-color: #aadfff; font-weight: 500' if c == country else '' for c in i], axis = 0)
        df_s.apply(lambda row: ['background-color: #ccebff;' if row.name == country else '' for cell in row], axis = 1)
        df_s.set_table_styles([{'selector': 'td:hover', 'props': [('background-color', '#ddfdff')]}])
        tt = {}
        for col in df.columns:
            tt[col] = 'Column median: ' + str(df.loc[:, col].median())
        df_s.set_tooltips(pd.DataFrame(tt, index = df.index))

        # Display a short descriptive title and the Dataframe.
        display(country + ' belongs to Cluster ' + str(cluster_number) + '. This Cluster contains a total of ' + str(df.shape[0]) + ' countries.')
        display(df_s)
    except Exception:
        return print('No indicators available for this country.')

@venn_group_out.capture(clear_output=True, wait=True)
def venn_group(cluster_df, country):
    try:
        _, econ_ind, eq_ind, socdem_ind = ind_dict.keys()       # NON ROBUST STATEMENT
        _, set_econ, set_eq, set_socdem = (set(cluster_df.loc[lambda df: df[ind] == df.loc[country, ind], ind].index.to_list()) for ind in ind_dict.keys())
        venn = venn3([set_econ, set_socdem, set_eq], (econ_ind, socdem_ind, eq_ind))

        venn.get_label_by_id('100').set_text('\n'.join(set_econ - set_socdem - set_eq)) # Only econ
        venn.get_label_by_id('010').set_text('\n'.join(set_socdem - set_econ - set_eq)) # Only socdem
        venn.get_label_by_id('001').set_text('\n'.join(set_eq - set_econ - set_socdem)) # Only eq

        # The three pair-intersections is guaranteed only if there is an intersection of the three groups.
        if len(set_econ & set_socdem & set_eq):
            venn.get_label_by_id('111').set_text('\n'.join(set_econ & set_socdem & set_eq))
            venn.get_label_by_id('110').set_text('\n'.join(set_econ & set_socdem - set_eq))
            venn.get_label_by_id('101').set_text('\n'.join(set_econ & set_eq - set_socdem))
            venn.get_label_by_id('011').set_text('\n'.join(set_socdem & set_eq - set_econ))
        else:
            # If no center, check the intersections that do exist.
            if len(set_econ & set_socdem - set_eq):
                venn.get_label_by_id('110').set_text('\n'.join(set_econ & set_socdem - set_eq))
            if len(set_econ & set_eq - set_socdem):
                venn.get_label_by_id('101').set_text('\n'.join(set_econ & set_eq - set_socdem))
            if len(set_socdem & set_eq - set_econ):
                venn.get_label_by_id('011').set_text('\n'.join(set_socdem & set_eq - set_econ))

        plt.rcParams["figure.figsize"] = (12, 12)
        plt.show()

    except:
        return print('No indicators available for this country.')

@chart_shift_ind_out.capture(clear_output=True, wait=True)
def chart_shift_ind(norm_df, mode, zone, indicator, shift):
    data_s = norm_df.loc[norm_df.index.get_level_values(mode) == zone, [col_gdp, indicator]].groupby(level = col_year).median()

    min_year_gdp = min_year + max(shift, 0)
    max_year_gdp = max_year + min(shift, 0)

    min_year_ind = min_year - min(shift, 0)
    max_year_ind = max_year - max(shift, 0)

    norm_gdp = norm(data_s.loc[min_year_gdp : max_year_gdp, [col_gdp]], None)
    norm_ind = norm(data_s.loc[min_year_ind : max_year_ind, [indicator]], None)

    plt.figure(figsize = (8,8))
    plt.plot(
                #norm_gdp.index.get_level_values(col_year),
                norm_gdp.reset_index(drop = True),
                color = "red", label = col_gdp)
    plt.plot(
                #norm_ind.index.get_level_values(col_year),
                norm_ind.reset_index(drop = True),
                color = "green", label = indicator)
    plt.legend(loc = "lower right")
    plt.show()

@chart_shift_corr_out.capture(clear_output=True, wait=True)
def chart_shift_corr(mode, zone, indicator):
    print(output_path)
    df_m = pd.read_csv(output_path + 'Shifted_Corr_' + mode + '.csv', index_col = [mode, col_shift])
    df_s = df_m.loc[df_m.index.get_level_values(mode) == zone, indicator]

    plt.figure(figsize = (8,8))
    plt.plot(
                df_s.index.get_level_values(col_shift),
                df_s.reset_index(drop = True),
                color = "green", label = indicator)
    plt.legend(loc = "lower right")
    plt.show()


In [48]:
import itertools
from scipy import stats

def styler_method(df, name, pvalue = None):    
        if pvalue == None:
                pvalue = 0.05
    #try:
        styles = [dict(selector="caption", props=[("background-color", "#98D3FF")])]
        left1 = pd.Series([pvalue], index=['P-value Spearman'])
        left2 = pd.Series([-1], index=['GDP Spearman Corr'])
        left3 = pd.Series([0], index=['GDP Spearman Corr'])
        dfs = df.style.highlight_between(left = left1, right = 1.5, axis = 1, props='color:white; background-color:red;')\
                .highlight_between(left = left2, right = 1.5, axis = 1, props='color:white; background-color:#929bfc;')\
                .highlight_between(left = left3, right = 1.5, axis = 1, props='color:white; background-color:#b3b9ff;')\
                .format('{:,.4f}', subset = ['GDP Spearman Corr'])\
                .format('{:,.12f}', subset = ['P-value Spearman']) \
                .set_caption(name).set_table_styles(styles)\
                .set_table_attributes("style='display:inline'")
    #except:
    #    dfs = 'No indicators have been found for the window dataframe in this range.'
        
        return dfs

def init_highest_table(indicators):    
    df_highest = pd.DataFrame(columns={"Indicator"})
    df_highest["Indicator"] = indicators
    df_highest["Year range"] = 0
    df_highest["Highest positive Spearman corr"] = 0
    df_highest["Year range "] = 0
    df_highest["Highest negative Spearman corr"] = 0
    df_highest.set_index("Indicator", inplace= True)

    return df_highest

def generate_years_combinations(min_diff: int, min, max):
    iterable = list(range(min, max + 1))
    iterable = list(itertools.combinations(iterable, 2))

    #The year length must be higher than 5. All entries with a lower range are deleted.
    for years in iterable.copy():
        if (years[1] - years[0]) < min_diff:
            iterable.remove(years)
    return iterable

In [49]:
@globe_corr_out.capture(clear_output=True, wait=True)
def globe_corr(df_corr, ind):
    N = 10
    fig = px.choropleth(df_corr, locations = df_corr.index, locationmode='country names', 
                        color = ind, projection="natural earth",
                        color_continuous_scale='RdBu',
                        width = 700, height=500)

    pos_corr = df_corr.drop(df_corr.columns.difference([ind]), axis = 1).sort_values(by = ind, axis = 0, ascending = False).head(n = N)
    neg_corr = df_corr.drop(df_corr.columns.difference([ind]), axis = 1).sort_values(by = ind, axis = 0, ascending = True).head(n = N)

    pos_corr = pos_corr.loc[pos_corr[ind] > 0]
    neg_corr = neg_corr.loc[neg_corr[ind] < 0]


    fig.update(layout_coloraxis_showscale=True)
    fig.show()
"""     
    pos_styler = pos_corr.style.set_table_attributes("style='display:inline'").set_caption('Direct correlation')
    neg_styler = neg_corr.style.set_table_attributes("style='display:inline'").set_caption('Inverse correlation')

    space = "\xa0" * 10
    display_html(pos_styler._repr_html_() + space  + neg_styler._repr_html_(), raw=True) """

# DELETEEEEEEE
# INCOMPLETE: REGION
@table_max_corr_country_out.capture(clear_output=True, wait=True)
def table_max_corr_country(mode, zone, threshold, pval):
    if mode == 'Region' and zone == 'Afghanistan': return None # CHANGE THIS FOR CHRIST SAKE
    print(mode, zone, threshold, pval)    
    df = search(threshold, mode, zone)
    # display(df)
    if df.empty:
        return print("No indicators have been found.")
    #df.reset_index()
    left1 = pd.Series([pval, pval], index=['P-value Pearson', 'P-value Spearman'])
    left2 = pd.Series([-1, -1], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    left3 = pd.Series([0, 0], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    df = df.style.highlight_between(left = left1, right = 1.5, axis = 1, props='color:white; background-color:red;')\
                 .highlight_between(left = left2, right = 1.5, axis = 1, props='color:white; background-color:#929bfc;')\
                 .highlight_between(left = left3, right = 1.5, axis = 1, props='color:white; background-color:#b3b9ff;')\
                 .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\
                 .format('{:,.12f}', subset = ['P-value Pearson', 'P-value Spearman']) 
    
    display(df)

@time_window_out.capture(clear_output=True, wait=True)
def time_window(df, mode, zone, years, threshold, pval):
    df_zone = df.loc[df.index.get_level_values(mode) == zone]
    #Load the selected year range and the global range.
    df_time = searchTimeSeries(threshold, years[0], years[1], True, df_zone)
    df_global = searchTimeSeries(threshold, years[0], years[1], False, df_zone)

    # Display Data
    if years[0] > years[1]: return print("Please, select a valid range of years.")
 
    space = "\xa0" * 10
    try:
        df_time = styler_method(df_time, str(years[0]) + '-' + str(years[1]), pval)._repr_html_()
    except Exception as e: 
        df_time = 'No indicators available for the selected parameters'
    
    try:
        df_global = styler_method(df_global, '2000-2020', pval)._repr_html_()
    except: 
        df_global = 'No indicators available for the selected parameters'
    
    display_html(df_time + space  + df_global, raw=True)

@table_high_out.capture(clear_output=True, wait=True)
def table_high(mode, zone, pval):
    if mode == 'Country':
        min_diff = 5
        
    elif mode == 'Region':
        min_diff = 2
    
    df_zone = df.loc[df.index.get_level_values(mode) == zone] #.drop(columns = 'GDP')
    indicators = df.columns
    df_highest = init_highest_table(indicators)

    i = 0
    computing_text = "Loading "
    print(computing_text, end="\r")
    
    #For all the combination of years...
    for years in generate_years_combinations(min_diff, min_year, max_year):
        i = (i + 1) % 50
        print (computing_text + "".join(["." for _ in range(i)]), end="\r")
        
        df_aux = searchTimeSeries(0, years[0], years[1], True, df_zone)
        #Delete indicators which are not available that year
        indicators_inter = list(set(indicators) & set(list(df_aux.index)))

        #For all the indicators availble that year....
        for indicator in indicators_inter:
            #Algorithm to search for the highest value
            indicator_corr_pos_last = df_highest[df_highest.index.get_level_values(0) == indicator]["Highest positive Spearman corr"][0]
            indicator_corr_neg_last = df_highest[df_highest.index.get_level_values(0) == indicator]["Highest negative Spearman corr"][0]
            
          
            indicator_corr_aux = df_aux[df_aux.index.get_level_values(0) == indicator]["GDP Spearman Corr"][0]
            indicator_p_value_aux = df_aux[df_aux.index.get_level_values(0) == indicator]["P-value Spearman"][0]

            if indicator_corr_aux != np.NaN and indicator_p_value_aux < pval:
                if indicator_corr_pos_last < indicator_corr_aux and indicator_corr_aux > 0:
                    df_highest.at[indicator, "Year range"] = str(years[0]) + '-' + str(years[1])
                    df_highest.at[indicator, "Highest positive Spearman corr"] = indicator_corr_aux
                elif indicator_corr_neg_last > indicator_corr_aux and indicator_corr_aux < 0:
                    df_highest.at[indicator, "Year range "] = str(years[0]) + '-' + str(years[1])
                    df_highest.at[indicator, "Highest negative Spearman corr"] = indicator_corr_aux

    df_highest = df_highest.replace(0, np.NaN).dropna(axis=0, how='all').fillna("-")
    
    print("                                                                                    ", end="\r")
    display(df_highest)

@plot_corr_ind_out.capture(clear_output=True, wait=True)
def plot_corr_ind(mode, zone, indicator: str, years: tuple):
    df_zone = df.loc[df.index.get_level_values(mode) == zone] 
    df_zone = get_years(df_zone, years)
    spear = stats.spearmanr(df_zone[indicator], df_zone['GDP'])
    df_zone = norm(df_zone, level = mode)

    #In case if by is region it groups by year.
    df_zone = df_zone.loc[:,['GDP', indicator]].groupby(level = 'Year').median()

    print(spear)
    plt.figure(figsize=(6,6))
    plt.plot(df_zone.index.get_level_values("Year"), df_zone["GDP"], color="red", label = 'GDP')
    plt.plot(df_zone.index.get_level_values("Year"), df_zone[indicator], color="green", label = indicator)
    plt.legend(loc="lower right")
    plt.show()


# WIDGETS

In [50]:
dropdown_mode = widgets.Dropdown(
    options = mode_list,
    description = 'Select: '
)

dropdown_zone = widgets.Dropdown(
    options = mode_dict[dropdown_mode.value],
    description = 'Show: '
)

dropdown_ind = widgets.Dropdown(
    options = indicators_list,
    description = 'Indicator: '
)

intslider_years = widgets.IntRangeSlider(
    value = [min_year, max_year],
    min = min_year,
    max = max_year,
    step = 1,
    description = 'Years: ',
)

intslider_shift = widgets.IntSlider(
    value = 0,
    min = min_year - max_year + 1,
    max = max_year - min_year - 1,
    step = 1,
    description = 'Shifts: '
)

floatslider_threshold = widgets.FloatSlider(
    value = 0.7,
    min = 0.0,
    max = 1.0,
    step = 0.05,
    description = 'Threshold: '
)

floatslider_confidence = widgets.FloatSlider(
    value = 0.95,
    min = 0.0,
    max = 1.0,
    step = 0.05,
    description = 'Confidence: '
)

In [51]:
def updatePlots(mode, zone, ind, years, shift, threshold, confidence):
    if ind != status_widget.ind:
        if mode == 'Region':
            status_widget.ind = None
            status_widget.ind_group = None
        else:
            status_widget.ind = ind
            group = group_of(ind)
            if status_widget.ind_group != group: status_widget.ind_group = group
    if mode != status_widget.mode:
        """ dropdown_zone.options = mode_dict[mode]
        status_widget.mode = mode
        data = None """
        status_widget.mode = 'Country'
    
    if zone != status_widget.zone:
        if mode == col_country:
            #print(status_widget.ind_group)
            #display(comp_dict[status_widget.ind_group])
            status_widget.comp_df = comp_dict[status_widget.ind_group].reset_index(col_cluster, drop = False)
        data = None
        status_widget.zone = zone

    if status_widget.years is None:
        status_widget.years = years

    if years[0] != status_widget.years[0]:
        status_widget.years[0]
    
    if years[1] != status_widget.years[1]:
        status_widget.years[1]

    if shift != status_widget.shift:
        status_widget.shift = shift
    
    if threshold != status_widget.threshold:
        status_widget.threshold = threshold
    
    if (1 - confidence) != status_widget.pval:
        status_widget.pval = (1 - confidence)
    
    cluster_number = cluster_dict[status_widget.ind_group].loc[cluster_dict[status_widget.ind_group].index == status_widget.zone, col_cluster].item()
    df_ind = cluster_dict[status_widget.ind_group]
    status_widget.ind_df = df_ind.loc[df_ind[col_cluster] == cluster_number].drop(col_cluster, axis = 'columns')

    plot_cluster_map (status_widget.comp_df)
    table_cluster (status_widget.ind_df, status_widget.ind_group, status_widget.zone)
    venn_group (status_widget.cluster_df, status_widget.zone)

    chart_shift_ind(status_widget.norm_df, status_widget.mode, status_widget.zone, status_widget.ind, status_widget.shift)
    chart_shift_corr(status_widget.mode, status_widget.zone, status_widget.ind)

    globe_corr(status_widget.corr_df, status_widget.ind)
    table_max_corr_country(status_widget.mode, status_widget.zone, status_widget.threshold, status_widget.pval)
    time_window(status_widget.df, status_widget.mode, status_widget.zone, status_widget.years, status_widget.threshold, status_widget.pval)
    table_high(status_widget.mode, status_widget.zone, status_widget.pval)
    plot_corr_ind(status_widget.mode, status_widget.zone, status_widget.ind, status_widget.years)

widgets.interact(updatePlots, mode = dropdown_mode, zone = dropdown_zone, ind = dropdown_ind,
                 years = intslider_years, shift = intslider_shift, threshold = floatslider_threshold, confidence = floatslider_confidence)
print()

interactive(children=(Dropdown(description='Select: ', options=('Country', 'Region'), value='Country'), Dropdo…